In [ ]:
!pip install -q opendatasets
import opendatasets as od
od.download(
    "https://www.kaggle.com/competitions/wind-turbine-damage-challenges")

Please provide your Kaggle credentials to download this dataset. Learn more: http://bit.ly/kaggle-creds
Your Kaggle username: visvaldas
Your Kaggle Key: ··········
Extracting archive ./wind-turbine-damage-challenges/wind-turbine-damage-challenges.zip to ./wind-turbine-damage-challenges


In [ ]:
!pip install -U openmim
!mim install mmengine
!pip install mmcv==2.2.0 -f https://download.openmmlab.com/mmcv/dist/cu121/torch2.4/index.html

Looking in links: https://download.openmmlab.com/mmcv/dist/cu124/torch2.6.0/index.html
  Using cached mmengine-0.10.7-py3-none-any.whl.metadata (20 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.8/46.8 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 452.7/452.7 kB 11.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 256.2/256.2 kB 25.5 MB/s eta 0:00:00
Looking in links: https://download.openmmlab.com/mmcv/dist/cu121/torch2.4/index.html
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.7/98.7 MB 6.6 MB/s eta 0:00:00


1. Создать датасет в mmdet/datasets, поменять classes и palette, задать импорт в init.
2. mmdetection/datasets/coco_detection - поменять пути, задать DefectDataset
3. mmdetection/config/... - поменять число классов с 80 на 5
4. mmdet - поменять макс. версию на 2.2.1
5. /usr/local/lib/...mmengine - 347 - загрузить веса

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# !cp /content/drive/MyDrive/mmdetection.zip /content/mmdetection.zip
!unzip mmdetection.zip
%cd mmdetection
!pip install -v -e .

Archive:  mmdetection.zip
   creating: mmdetection/
   creating: mmdetection/.circleci/
  inflating: mmdetection/.circleci/config.yml  
   creating: mmdetection/.circleci/docker/
  inflating: mmdetection/.circleci/docker/Dockerfile  
  inflating: mmdetection/.circleci/test.yml  
   creating: mmdetection/.dev_scripts/
  inflating: mmdetection/.dev_scripts/batch_test_list.py  
  inflating: mmdetection/.dev_scripts/batch_train_list.txt  
  inflating: mmdetection/.dev_scripts/benchmark_filter.py  
  inflating: mmdetection/.dev_scripts/benchmark_full_models.txt  
  inflating: mmdetection/.dev_scripts/benchmark_inference_fps.py  
  inflating: mmdetection/.dev_scripts/benchmark_options.py  
  inflating: mmdetection/.dev_scripts/benchmark_test.py  
  inflating: mmdetection/.dev_scripts/benchmark_test_image.py  
  inflating: mmdetection/.dev_scripts/benchmark_train.py  
  inflating: mmdetection/.dev_scripts/benchmark_train_models.txt  
  inflating: mmdetection/.dev_scripts/benchmark_valid_flops

In [ ]:
!python mmdetection/tools/train.py /content/mmdetection/configs/faster_rcnn/faster-rcnn_r101_fpn_1x_coco.py

In [ ]:
# !cp '/content/drive/MyDrive/epoch_12 (1).pth' /content/epoch_12.pth

In [ ]:
!python tools/test.py configs/faster_rcnn/faster-rcnn_r101_fpn_1x_coco.py /content/epoch_12.pth --out /content/result.pkl --show --show-dir /content/show

In [ ]:
import glob
dataset = glob.glob("/content/wind-turbine-damage-challenges/Wind Turbine damage/test/*.jpg")

In [ ]:
from mmdet.apis import DetInferencer

config_file = '/content/mmdetection/configs/faster_rcnn/faster-rcnn_r101_fpn_1x_coco.py'
checkpoint_file = '/content/epoch_12.pth'

inferencer = DetInferencer(model=config_file, weights=checkpoint_file, device='cuda:0')
results = inferencer('/content/wind-turbine-damage-challenges/Wind Turbine damage/test/')

Loads checkpoint by local backend from path: /content/epoch_12.pth


Output()

In [ ]:
results['predictions']

In [ ]:
import pickle


with open('result.pkl', 'rb') as f:
    data = pickle.load(f)

In [ ]:
data

In [ ]:
!pip install ultralytics

In [ ]:
import os
import json
from tqdm import tqdm

def coco_to_yolo(coco_path, output_dir, images_dir):
    with open(coco_path) as f:
        coco = json.load(f)

    os.makedirs(output_dir, exist_ok=True)

    # Создаём словарь id → category_id
    category_map = {cat['id']: idx for idx, cat in enumerate(coco['categories'])}

    # Создаём словарь image_id → file_name, size
    image_info = {img['id']: img for img in coco['images']}

    for ann in tqdm(coco['annotations']):
        img = image_info[ann['image_id']]
        width, height = img['width'], img['height']
        x, y, w, h = ann['bbox']
        xc = (x + w / 2) / width
        yc = (y + h / 2) / height
        w /= width
        h /= height
        class_id = category_map[ann['category_id']]

        label_path = os.path.join(output_dir, os.path.splitext(img['file_name'])[0] + ".txt")
        with open(label_path, 'a') as f:
            f.write(f"{class_id} {xc:.6f} {yc:.6f} {w:.6f} {h:.6f}\n")


In [ ]:
coco_to_yolo('/content/wind-turbine-damage-challenges/Wind Turbine damage/train/_annotations.coco.json', '/content/datasets/wind-turbine-damage-challenges/labels/train', '/content/wind-turbine-damage-challenges/Wind Turbine damage/train/')

100%|██████████| 9150/9150 [00:00<00:00, 40925.71it/s]


In [ ]:
coco_to_yolo('/content/wind-turbine-damage-challenges/Wind Turbine damage/valid/_annotations.coco.json', '/content/datasets/wind-turbine-damage-challenges/labels/valid', '/content/wind-turbine-damage-challenges/Wind Turbine damage/valid/')

100%|██████████| 2619/2619 [00:00<00:00, 28029.17it/s]


In [ ]:
!cp -r 'wind-turbine-damage-challenges/Wind Turbine damage/train' '/content/datasets/wind-turbine-damage-challenges/images/train'

In [ ]:
!cp -r 'wind-turbine-damage-challenges/Wind Turbine damage/valid' '/content/datasets/wind-turbine-damage-challenges/images/val'

In [ ]:
!cp -r 'wind-turbine-damage-challenges/Wind Turbine damage/test' '/content/datasets/wind-turbine-damage-challenges/images/test'

In [ ]:
from ultralytics import YOLO

model = YOLO('yolov8n.pt')  # или yolov8s.pt
model.train(data='dataset.yaml', epochs=50, imgsz=640)


In [ ]:
import pandas as pd

results = model.predict(source='datasets/wind-turbine-damage-challenges/images/test', save=False)

rows = []
for r in results:
    image_id = os.path.basename(r.path)
    for box in r.boxes:
        class_id = int(box.cls)
        conf = float(box.conf)
        x1, y1, x2, y2 = box.xyxy[0].tolist()
        rows.append([image_id, x1, y1, x2, y2, class_id, conf])

df = pd.DataFrame(rows, columns=['image_id', 'x1', 'y1', 'x2', 'y2', 'class_id', 'confidence'])
df.to_csv("submission.csv", index=False)


In [ ]:
!zip -r * content.zip

In [ ]:
!cp datasets.zip drive/MyDrive

In [ ]:
import json
import pandas as pd

# Путь к файлу с результатами инференса
results_path = "farazlar.txt"  # или .json, если вы сохраняете в этом формате
output_csv = "submission.csv"

# Загрузка результатов
with open(results_path, "r", encoding="utf-8") as f:
    results = json.load(f)

# Составляем submission
submission_data = []

i = 0

for image_id, result in enumerate(results):
    labels = result.get("labels", [])
    scores = result.get("scores", [])
    bboxes = result.get("bboxes", [])

    for label, score, bbox in zip(labels, scores, bboxes):
        xmin, ymin, xmax, ymax = bbox
        submission_data.append({
            "image_id": image_id,
            "xmin": round(xmin, 2),
            "ymin": round(ymin, 2),
            "xmax": round(xmax, 2),
            "ymax": round(ymax, 2),
            "label": label,
            "score": round(score, 4)
        })

# Преобразуем в DataFrame и сохраняем
df = pd.DataFrame(submission_data)
df.to_csv(output_csv, index=False)

print(f"Сабмит сохранён в {output_csv}")

Сабмит сохранён в submission.csv


In [ ]:
len(results)

151